In [1]:
import mechanize
import cookielib
import BeautifulSoup

br = mechanize.Browser()

# Cookie Jar
cj = cookielib.LWPCookieJar()
br.set_cookiejar(cj)

# Browser options
br.set_handle_equiv(True)
br.set_handle_gzip(True)
br.set_handle_redirect(True)
br.set_handle_referer(True)
br.set_handle_robots(False)

br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]


In [2]:
def getDateTimeVals(soup):
    vals = soup.findAll('td',attrs={"width":"10%"})
    vals = vals[1:]
    times = soup.findAll('td',attrs={"width":"7%"})
    times = times[2:]
    times = times[::2]
    dates = soup.findAll('td',attrs={"width":"6%"})
    dates = dates[1:]
    dates= map(lambda x: x.text.split(';')[1],dates)
    times = map(lambda x: x.text.split(';')[1],times)
    vals = map(lambda x: x.text, vals)
    return dates,times,vals

In [3]:
import time
def getBrowser(pollutant,station):
    br = mechanize.Browser()

    # Cookie Jar
    cj = cookielib.LWPCookieJar()
    br.set_cookiejar(cj)

    # Browser options
    br.set_handle_equiv(True)
    br.set_handle_gzip(True)
    br.set_handle_redirect(True)
    br.set_handle_referer(True)
    br.set_handle_robots(False)

    br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

    br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]
    
    br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)
    br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]
    br.open('http://www.cpcb.gov.in/CAAQM/frmUserAvgReportCriteria.aspx')
    br.select_form(nr=0)
    br.form['ddlState'] = ['6']
    br.submit()
    br.select_form(nr=0)
    br.form['ddlCity'] = ['85']
    br.submit()
    br.select_form(nr=0)
    br.form['ddlCriteria'] = ['1']
    br.form['ddlStation'] = [station.name]
    br.submit()
    br.select_form(nr=0)
    br.form['lstBoxChannelLeft'] = [pollutant.name]
    br.submit(name='btnAdd')
    br.select_form(nr=0)
    return br
    
def getStations():
    br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)
    br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]
    br.open('http://www.cpcb.gov.in/CAAQM/frmUserAvgReportCriteria.aspx')
    br.select_form(nr=0)
    br.form['ddlState'] = ['6']
    br.submit()
    br.select_form(nr=0)
    br.form['ddlCity'] = ['85']
    br.submit()
    br.select_form(nr=0)
    return br.form.controls[7].get_items(),br


def getPollutants(br,station):
    br.form['ddlStation'] = [station.name]
    br.submit()
    br.select_form(nr=0)
    return br.form.controls[8].get_items(),br

def getResponse(br,station,pollutant):
    while True:
        try:
            
            print("Sending request..")
            response = br.submit(name='btnSubmit')
            return response
        except:
            print("Trying ..")
            freshBr = getBrowser(pollutant,station)
            br = freshBr
            

def getPollData(pollutant,br,startDate,endDate,station):
    br.form['txtDateFrom'] = startDate.strftime("%d/%m/%Y")
    br.form['txtDateTo'] = endDate.strftime("%d/%m/%Y")
    response = getResponse(br,station,pollutant)
    soup = BeautifulSoup.BeautifulSoup(response.read())
    return getDateTimeVals(soup)

In [4]:
import os
import datetime
import pandas as pd
from IPython.display import clear_output

print("Getting stations...")
stations,br = getStations()
for station in stations[1:]:
    clear_output()
    stationName = station.attrs['label']
    if not os.path.isdir(stationName):
        os.mkdir(stationName)
    print("Getting pollutants for "+ stationName)
    pollutants,br = getPollutants(br,station)
    br.form['ddlCriteria'] = ['1']
    for pollutant in pollutants[1:]:
        
        print(pollutant.attrs['label'])
        pollutantName = pollutant.attrs['label']
        if pollutantName[:28] == 'Suspended Particulate Matter':
            pollutantName = 'Suspended Particulate Matter'
        fileName = stationName + '/' +pollutantName+ '.csv'
        logName = stationName + '/' +pollutantName + '.log'
        startDate = datetime.date(2011,1,1)
        dateCol = []
        timeCol = []
        valuesCol = []
        csv = pd.DataFrame({"Time":timeCol,"Date":dateCol,"Values":valuesCol})
        
        if os.path.isfile(fileName):
            with open(logName) as file:
                date_ = file.readlines()[0]
                startDate = datetime.datetime.strptime(date_, "%d/%m/%Y").date()
            csv = pd.read_csv(fileName)
            csv = csv.fillna("NA")
            csv = csv.as_matrix()
            dateCol = csv[:,0].tolist()
            timeCol = csv[:,1].tolist()
            valuesCol = csv[:,2].tolist()
        
        if startDate >= datetime.date.today():
            continue
        
        br.form['lstBoxChannelLeft'] = [pollutant.name]
        br.submit(name='btnAdd')
        br.select_form(nr=0)      
        
        while(startDate< datetime.date.today()):
#             print("Start Date:{0}".format(startDate))
            start_time = time.time()

            endDate = startDate + datetime.timedelta(90)
            
            dates,times,vals = getPollData(pollutant,br,startDate,endDate,station)
            
            valuesCol.extend(vals)
            dateCol.extend(dates)
            timeCol.extend(times)
            
            
            br.back()
            br.select_form(nr=0)
            with open(logName,'w') as file_:
                file_.write(endDate.strftime("%d/%m/%Y"))
            csv = pd.DataFrame({"Time":timeCol,"Date":dateCol,"Values":valuesCol})
            csv.to_csv(fileName,index=False)
            
            elapsed_time = time.time() - start_time
            print("Got {0} data from {1} to {2} in {3}".format(len(dates),startDate,endDate,elapsed_time))
            startDate = endDate
        br.form['lstBoxChannelRight'] = [pollutant.name]
        br.submit(name='btnRemove')
        br.select_form(nr=0)
        
        
    

Getting pollutants for Noida Sector 62
